## **Preparando base geográfica**
***
##### **Objetivo**: criar uma tabela com a relação de todas as cidades do país e seus respectivos estados e regiões; e armazená-la no banco de dados do projeto.
##### **Resultado**: criação de um código que lê a base geográfica a partir de um arquivo '.csv' e que complementa informações de Estados e Regiões a partir de uma tabela contida em um site; após tratamento das informações, armazena a tabela final no banco de dados do projeto ('dados.db')
***
###### O arquivo .csv foi gerado a partir do arquivo 'RELATORIO_DTB_BRASIL_MUNICIPIO.xls', que está contido em uma pasta zipada baixada diretamente do site do IBGE. Adiante segue link para download da mesma: https://geoftp.ibge.gov.br/organizacao_do_territorio/estrutura_territorial/divisao_territorial/2021/DTB_2021.zip

In [40]:
# Bibliotecas utilizadas
import pandas  as pd
import sqlite3 as sql3

### Consultando DataFrame principal

In [41]:
# Registrando colunas a serem selecionadas do arquivo .csv
colunas = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

# Coletando base geográfica do arquivo .csv
df = pd.read_csv( 'ibge_original.csv',
                  sep = ';',
                  usecols = colunas,
                  dtype = str )

In [42]:
# Renomeando colunas da tabela
df.rename( columns = { 'UF'                                  : 'id_uf',
                       'Nome_UF'                             : 'nome_uf',
                       'Região Geográfica Intermediária'     : 'id_regiao_geografica_intermediaria',
                       'Nome Região Geográfica Intermediária': 'nome_regiao_geografica_intermediaria',
                       'Região Geográfica Imediata'          : 'id_regiao_geografica_imediata',
                       'Nome Região Geográfica Imediata'     : 'nome_regiao_geografica_imediata',
                       'Mesorregião Geográfica'              : 'id_mesorregiao_geografica',
                       'Nome_Mesorregião'                    : 'nome_mesorregiao',
                       'Microrregião Geográfica'             : 'id_microrregiao_geografica',
                       'Nome_Microrregião'                   : 'nome_microrregiao_geografica',
                       'Município'                           : 'id_municipio_curto',
                       'Código Município Completo'           : 'id_municipio_completo',
                       'Nome_Município'                      : 'nome_municipio' },
           inplace = True )

In [43]:
# Visualizando tabela
df.head()

,id_uf,nome_uf,id_regiao_geografica_intermediaria,nome_regiao_geografica_intermediaria,id_regiao_geografica_imediata,nome_regiao_geografica_imediata,id_mesorregiao_geografica,nome_mesorregiao,id_microrregiao_geografica,nome_microrregiao_geografica,id_municipio_curto,id_municipio_completo,nome_municipio
0,11,Rondônia,1102,Ji-Paraná,110005,Cacoal,02,Leste Rondoniense,006,Cacoal,00015,1100015,Alta Floresta D'Oeste
1,11,Rondônia,1102,Ji-Paraná,110005,Cacoal,02,Leste Rondoniense,006,Cacoal,00379,1100379,Alto Alegre dos Parecis
2,11,Rondônia,1101,Porto Velho,110002,Ariquemes,02,Leste Rondoniense,003,Ariquemes,00403,1100403,Alto Paraíso
3,11,Rondônia,1102,Ji-Paraná,110004,Ji-Paraná,02,Leste Rondoniense,005,Alvorada D'Oeste,00346,1100346,Alvorada D'Oeste
4,11,Rondônia,1101,Porto Velho,110002,Ariquemes,02,Leste Rondoniense,003,Ariquemes,00023,1100023,Ariquemes


### Consultando DataFrame complementar
Coletando tabela com informações adicionais dos Estados, como Sigla e Região

In [44]:
# Coletando tabela do site que possui os dados complementares
dadosComp = pd.read_html( 'https://www.estadosecapitaisdobrasil.com/' )[0]

In [45]:
# Visualizando tabela
dadosComp.head()

,Bandeira,Estado,Sigla,Capital,Região
0,NaN,Acre,AC,Rio Branco,Norte
1,NaN,Alagoas,AL,Maceió,Nordeste
2,NaN,Amapá,AP,Macapá,Norte
3,NaN,Amazonas,AM,Manaus,Norte
4,NaN,Bahia,BA,Salvador,Nordeste


In [46]:
# Selecionando apenas colunas de interesse
dadosComp = dadosComp.iloc[ :, [1, 2, 4] ]

In [47]:
# Renomeando colunas
dadosComp.rename( columns = { 'Estado': 'nome_uf',
                              'Sigla' : 'sigla_uf',
                              'Região': 'regiao_uf' },
                  inplace = True )

In [48]:
# Visualizando tabela
dadosComp.head()

,nome_uf,sigla_uf,regiao_uf
0,Acre,AC,Norte
1,Alagoas,AL,Nordeste
2,Amapá,AP,Norte
3,Amazonas,AM,Norte
4,Bahia,BA,Nordeste


### Unificando DataFrame principal com o complementar

In [49]:
# Unificando o df com as informações complementares de Estados e Regiões
df = df.merge( dadosComp, how = 'left', on  = 'nome_uf' )

In [50]:
# Reordenando as colunas do DataFrame
df = df.iloc[ :, [0, 1, 13, 14, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] ]

In [51]:
# Visualizando tabela
df.head()

,id_uf,nome_uf,sigla_uf,regiao_uf,id_regiao_geografica_intermediaria,nome_regiao_geografica_intermediaria,id_regiao_geografica_imediata,nome_regiao_geografica_imediata,id_mesorregiao_geografica,nome_mesorregiao,id_microrregiao_geografica,nome_microrregiao_geografica,id_municipio_curto,id_municipio_completo,nome_municipio
0,11,Rondônia,RO,Norte,1102,Ji-Paraná,110005,Cacoal,02,Leste Rondoniense,006,Cacoal,00015,1100015,Alta Floresta D'Oeste
1,11,Rondônia,RO,Norte,1102,Ji-Paraná,110005,Cacoal,02,Leste Rondoniense,006,Cacoal,00379,1100379,Alto Alegre dos Parecis
2,11,Rondônia,RO,Norte,1101,Porto Velho,110002,Ariquemes,02,Leste Rondoniense,003,Ariquemes,00403,1100403,Alto Paraíso
3,11,Rondônia,RO,Norte,1102,Ji-Paraná,110004,Ji-Paraná,02,Leste Rondoniense,005,Alvorada D'Oeste,00346,1100346,Alvorada D'Oeste
4,11,Rondônia,RO,Norte,1101,Porto Velho,110002,Ariquemes,02,Leste Rondoniense,003,Ariquemes,00023,1100023,Ariquemes


### Inserindo resultado no banco de dados do projeto

In [ ]:
# Criando / conectando-se com o banco de dados do projeto
conn = sql3.connect( '../analise_dados/dados.db' )

# Armazenando a tabela tratada no banco de dados do projeto
df.to_sql( 'geografia',
           con       = conn,
           if_exists = 'replace',
           index     = False )